# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [39]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [40]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.describe()

from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [41]:
symbol = 'AAPL'

baseUrl = f'https://sandbox.iexapis.com/stable'

quoteUrl = f'/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
apiUrl = baseUrl + quoteUrl

data = requests.get(apiUrl).json()
data['latestPrice']

131.79

In [42]:
symbol = 'AAPL'

statsUrl = f'/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
apiUrl = baseUrl + statsUrl

data = requests.get(apiUrl).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2216982378738,
 'week52high': 190.77,
 'week52low': 131.2,
 'week52highSplitAdjustOnly': 191.62,
 'week52lowSplitAdjustOnly': 134.23,
 'week52change': 0.014846837103073924,
 'sharesOutstanding': 16815340009,
 'float': 0,
 'avg10Volume': 109370182,
 'avg30Volume': 95091254,
 'day200MovingAvg': 169.48,
 'day50MovingAvg': 146.58,
 'employees': 153504,
 'ttmEPS': 6.42,
 'ttmDividendRate': 0.890976313583052,
 'dividendYield': 0.006869006332736444,
 'nextDividendDate': '',
 'exDividendDate': '2022-05-05',
 'nextEarningsDate': '2022-07-16',
 'peRatio': 21.07113837274434,
 'beta': 1.3307148597599563,
 'maxChangePercent': 49.79258063916449,
 'year5ChangePercent': 2.86485983546818,
 'year2ChangePercent': 0.544810049750926,
 'year1ChangePercent': 0.014364457769661235,
 'ytdChangePercent': -0.2640873079335699,
 'month6ChangePercent': -0.234685425900497,
 'month3ChangePercent': -0.1966292075702293,
 'month1ChangePercent': -0.0440449544172364,
 'day30Change

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [43]:
data['year1ChangePercent']

0.014364457769661235

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [44]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbolGroups = list(chunks(stocks['Ticker'], 100))
symbolStrings = []
for i in range(0, len(symbolGroups)):
    symbolStrings.append(','.join(symbolGroups[i]))
#     print(symbolStrings[i])

myColumns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [45]:
finalDataFrame = pd.DataFrame(columns = myColumns)

for symbolString in symbolStrings:
    batchApiUrlCall = baseUrl + f'/stock/market/batch?symbols={symbolString}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchApiUrlCall).json()
    for symbol in symbolString.split(','):
        finalDataFrame = finalDataFrame.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index = myColumns
            ),
            ignore_index = True
        )

finalDataFrame

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,116.50,-0.222780,N/A
1,AAL,13.10,-0.435611,N/A
2,AAP,173.07,-0.115203,N/A
3,AAPL,137.49,0.014435,N/A
4,ABBV,138.52,0.276415,N/A
...,...,...,...,...
499,YUM,109.50,-0.028685,N/A
500,ZBH,105.82,-0.362541,N/A
501,ZBRA,289.23,-0.423505,N/A
502,ZION,53.53,0.038971,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [46]:
finalDataFrame.sort_values(
    'One-Year Price Return',
    ascending = False, 
    inplace = True 
)
finalDataFrame = finalDataFrame[:50]
finalDataFrame.reset_index(inplace = True, drop = True)

# finalDataFrame = finalDataFrame.copy()
finalDataFrame

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,58.38,1.369204,N/A
1,OXY,57.03,1.041213,N/A
2,MRO,25.28,0.977643,N/A
3,APA,39.67,0.942452,N/A
4,CF,89.16,0.821364,N/A
5,CTRA,26.79,0.767446,N/A
6,MOS,51.60,0.733080,N/A
7,COP,94.74,0.710670,N/A
8,MCK,314.94,0.645670,N/A
9,VLO,117.65,0.560477,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [47]:
def enterPortfolioSize():
    global portfolioSize
    portfolioSize = input('Enter the value of your portfolio:')

    try:
        portfolioSize = float(portfolioSize)
    except ValueError:
        print('That is not an integer!\nPlease try again')
        portfolioSize = input('Enter the value of your portfolio:')
        portfolioSize = float(portfolioSize)

enterPortfolioSize()
# print(portfolioSize)

Enter the value of your portfolio:987654321


In [48]:
positionSize = portfolioSize / len(finalDataFrame.index)
print(positionSize)
for i in range(0, len(finalDataFrame.index)):
#     print(finalDataFrame.loc[i, 'Number of Shares to Buy'])
    
#     finalDataFrame.iloc[i, 3] = math.floor(positionSize / finalDataFrame.iloc[i, 1])
    finalDataFrame.loc[i, 'Number of Shares to Buy'] = math.floor(positionSize / finalDataFrame.loc[i, 'Price'])

finalDataFrame

19753086.42


/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,58.38,1.369204,338353
1,OXY,57.03,1.041213,346363
2,MRO,25.28,0.977643,781372
3,APA,39.67,0.942452,497935
4,CF,89.16,0.821364,221546
5,CTRA,26.79,0.767446,737330
6,MOS,51.60,0.733080,382811
7,COP,94.74,0.710670,208497
8,MCK,314.94,0.645670,62720
9,VLO,117.65,0.560477,167897


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [71]:
hqmColumns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Price Return Percentiles',
    'Six-Month Price Return',
    'Six-Month Price Return Percentiles',
    'Three-Month Price Return',
    'Three-Month Price Return Percentiles',
    'One-Month Price Return',
    'One-Month Price Return Percentiles',
    'HQM Score',
]

hqmDataFrame = pd.DataFrame(columns = hqmColumns)

for symbolString in symbolStrings:
    batchApiUrlCall = baseUrl + f'/stock/market/batch?symbols={symbolString}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchApiUrlCall).json()
    for symbol in symbolString.split(','):
        hqmDataFrame = hqmDataFrame.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A',
                ],
                index = hqmColumns
            ),
            ignore_index = True
        )


hqmDataFrame

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Return Percentiles,Six-Month Price Return,Six-Month Price Return Percentiles,Three-Month Price Return,Three-Month Price Return Percentiles,One-Month Price Return,One-Month Price Return Percentiles,HQM Score
0,A,115.32,N/A,-0.227565,N/A,-0.265965,N/A,-0.189705,N/A,-0.080128,N/A,N/A
1,AAL,13.53,N/A,-0.420202,N/A,-0.248001,N/A,-0.232516,N/A,-0.234301,N/A,N/A
2,AAP,170.66,N/A,-0.114880,N/A,-0.270979,N/A,-0.183799,N/A,-0.140081,N/A,N/A
3,AAPL,136.00,N/A,0.014348,N/A,-0.232642,N/A,-0.198424,N/A,-0.043226,N/A,N/A
4,ABBV,145.05,N/A,0.281583,N/A,0.092176,N/A,-0.129781,N/A,-0.089806,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,109.70,N/A,-0.028511,N/A,-0.168224,N/A,-0.098251,N/A,0.000684,N/A,N/A
500,ZBH,105.26,N/A,-0.357535,N/A,-0.151193,N/A,-0.153776,N/A,-0.123037,N/A,N/A
501,ZBRA,302.81,N/A,-0.419348,N/A,-0.528580,N/A,-0.341769,N/A,-0.110132,N/A,N/A
502,ZION,53.27,N/A,0.040426,N/A,-0.147002,N/A,-0.239883,N/A,0.003844,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [72]:
timePeriods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]

for row in hqmDataFrame.index:
    for timePeriod in timePeriods:
        priceReturnColumn = f'{timePeriod} Price Return'
        percentilesColumn = f'{timePeriod} Price Return Percentiles'
        hqmDataFrame.loc[row, percentilesColumn] = score(
            hqmDataFrame[priceReturnColumn],
            hqmDataFrame.loc[row, priceReturnColumn]
        ) / 100
        
hqmDataFrame

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Return Percentiles,Six-Month Price Return,Six-Month Price Return Percentiles,Three-Month Price Return,Three-Month Price Return Percentiles,One-Month Price Return,One-Month Price Return Percentiles,HQM Score
0,A,115.32,N/A,-0.227565,0.265873,-0.265965,0.323413,-0.189705,0.430556,-0.080128,0.444444,N/A
1,AAL,13.53,N/A,-0.420202,0.069444,-0.248001,0.367063,-0.232516,0.269841,-0.234301,0.011905,N/A
2,AAP,170.66,N/A,-0.114880,0.47619,-0.270979,0.303571,-0.183799,0.460317,-0.140081,0.119048,N/A
3,AAPL,136.00,N/A,0.014348,0.708333,-0.232642,0.396825,-0.198424,0.396825,-0.043226,0.694444,N/A
4,ABBV,145.05,N/A,0.281583,0.946429,0.092176,0.89881,-0.129781,0.619048,-0.089806,0.390873,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,109.70,N/A,-0.028511,0.634921,-0.168224,0.541667,-0.098251,0.696429,0.000684,0.890873,N/A
500,ZBH,105.26,N/A,-0.357535,0.121032,-0.151193,0.565476,-0.153776,0.561508,-0.123037,0.170635,N/A
501,ZBRA,302.81,N/A,-0.419348,0.071429,-0.528580,0.027778,-0.341769,0.05754,-0.110132,0.248016,N/A
502,ZION,53.27,N/A,0.040426,0.748016,-0.147002,0.575397,-0.239883,0.25,0.003844,0.900794,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [73]:
from statistics import mean

for row in hqmDataFrame.index:
    momentumPercentile = []
    for timePeriod in timePeriods:
        percentilesColumn = f'{timePeriod} Price Return Percentiles'
        momentumPercentile.append(
            hqmDataFrame.loc[row, percentilesColumn]
        )
        hqmDataFrame.loc[row, 'HQM Score'] = mean(momentumPercentile)
        
hqmDataFrame

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Return Percentiles,Six-Month Price Return,Six-Month Price Return Percentiles,Three-Month Price Return,Three-Month Price Return Percentiles,One-Month Price Return,One-Month Price Return Percentiles,HQM Score
0,A,115.32,N/A,-0.227565,0.265873,-0.265965,0.323413,-0.189705,0.430556,-0.080128,0.444444,0.366071
1,AAL,13.53,N/A,-0.420202,0.069444,-0.248001,0.367063,-0.232516,0.269841,-0.234301,0.011905,0.179563
2,AAP,170.66,N/A,-0.114880,0.47619,-0.270979,0.303571,-0.183799,0.460317,-0.140081,0.119048,0.339782
3,AAPL,136.00,N/A,0.014348,0.708333,-0.232642,0.396825,-0.198424,0.396825,-0.043226,0.694444,0.549107
4,ABBV,145.05,N/A,0.281583,0.946429,0.092176,0.89881,-0.129781,0.619048,-0.089806,0.390873,0.71379
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,109.70,N/A,-0.028511,0.634921,-0.168224,0.541667,-0.098251,0.696429,0.000684,0.890873,0.690972
500,ZBH,105.26,N/A,-0.357535,0.121032,-0.151193,0.565476,-0.153776,0.561508,-0.123037,0.170635,0.354663
501,ZBRA,302.81,N/A,-0.419348,0.071429,-0.528580,0.027778,-0.341769,0.05754,-0.110132,0.248016,0.10119
502,ZION,53.27,N/A,0.040426,0.748016,-0.147002,0.575397,-0.239883,0.25,0.003844,0.900794,0.618552


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [74]:
hqmDataFrame.sort_values(
    'HQM Score',
    ascending = False,
    inplace = True,
)
hqmDataFrame = hqmDataFrame[:50]
hqmDataFrame.reset_index(inplace = True, drop = True)
hqmDataFrame

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Return Percentiles,Six-Month Price Return,Six-Month Price Return Percentiles,Three-Month Price Return,Three-Month Price Return Percentiles,One-Month Price Return,One-Month Price Return Percentiles,HQM Score
0,VRTX,270.10,N/A,0.402383,0.962302,0.228975,0.958333,0.056414,0.964286,0.030747,0.950397,0.958829
1,WRB,68.43,N/A,0.400979,0.960317,0.258839,0.960317,0.046189,0.956349,0.008142,0.916667,0.948413
2,LLY,303.70,N/A,0.362975,0.954365,0.097454,0.906746,0.014947,0.938492,0.016619,0.934524,0.933532
3,DLTR,155.28,N/A,0.517713,0.974206,0.105891,0.914683,-0.045317,0.85119,0.111241,0.990079,0.93254
4,NOC,462.80,N/A,0.229081,0.93254,0.193245,0.948413,0.043077,0.954365,-0.006627,0.869048,0.926091
5,ULTA,407.76,N/A,0.217285,0.926587,0.039975,0.847222,0.012130,0.93254,0.126991,0.994048,0.925099
6,AZO,2066.00,N/A,0.457230,0.970238,-0.001934,0.805556,0.021256,0.946429,0.056693,0.978175,0.925099
7,CEG,59.00,N/A,0.106473,0.853175,0.104358,0.912698,0.188426,0.996032,0.015386,0.930556,0.923115
8,HSY,213.17,N/A,0.212754,0.920635,0.101075,0.910714,-0.003393,0.906746,0.004632,0.902778,0.910218
9,APA,39.48,N/A,0.929097,0.994048,0.636029,0.996032,0.005764,0.926587,-0.041028,0.718254,0.90873


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [75]:
enterPortfolioSize()

Enter the value of your portfolio:10000000


In [76]:
positionSize = float(portfolioSize / len(hqmDataFrame.index))
for i in hqmDataFrame.index:
    hqmDataFrame.loc[i, 'Number of Shares to Buy'] = math.floor(positionSize / hqmDataFrame.loc[i, 'Price'])
    
hqmDataFrame

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Return Percentiles,Six-Month Price Return,Six-Month Price Return Percentiles,Three-Month Price Return,Three-Month Price Return Percentiles,One-Month Price Return,One-Month Price Return Percentiles,HQM Score
0,VRTX,270.10,740,0.402383,0.962302,0.228975,0.958333,0.056414,0.964286,0.030747,0.950397,0.958829
1,WRB,68.43,2922,0.400979,0.960317,0.258839,0.960317,0.046189,0.956349,0.008142,0.916667,0.948413
2,LLY,303.70,658,0.362975,0.954365,0.097454,0.906746,0.014947,0.938492,0.016619,0.934524,0.933532
3,DLTR,155.28,1287,0.517713,0.974206,0.105891,0.914683,-0.045317,0.85119,0.111241,0.990079,0.93254
4,NOC,462.80,432,0.229081,0.93254,0.193245,0.948413,0.043077,0.954365,-0.006627,0.869048,0.926091
5,ULTA,407.76,490,0.217285,0.926587,0.039975,0.847222,0.012130,0.93254,0.126991,0.994048,0.925099
6,AZO,2066.00,96,0.457230,0.970238,-0.001934,0.805556,0.021256,0.946429,0.056693,0.978175,0.925099
7,CEG,59.00,3389,0.106473,0.853175,0.104358,0.912698,0.188426,0.996032,0.015386,0.930556,0.923115
8,HSY,213.17,938,0.212754,0.920635,0.101075,0.910714,-0.003393,0.906746,0.004632,0.902778,0.910218
9,APA,39.48,5065,0.929097,0.994048,0.636029,0.996032,0.005764,0.926587,-0.041028,0.718254,0.90873


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [81]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqmDataFrame.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [82]:
backgroundColor = '#0a0a23'
fontColor = '#ffffff'

stringTemplate = writer.book.add_format(
        {
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

dollarTemplate = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

integerTemplate = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

percentTemplate = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

In [83]:
columnFormats = {
    'A': ['Ticker', stringTemplate],
    'B': ['Price', dollarTemplate],
    'C': ['Number of Shares to Buy', integerTemplate],
    'D': ['One-Year Price Return', percentTemplate],
    'E': ['One-Year Price Return Percentiles', percentTemplate],
    'F': ['Six-Month Price Return', percentTemplate],
    'G': ['Six-Month Price Return Percentiles', percentTemplate],
    'H': ['Three-Month Price Return', percentTemplate],
    'I': ['Three-Month Price Return Percentiles', percentTemplate],
    'J': ['One-Month Price Return', percentTemplate],
    'K': ['One-Month Price Return Percentiles', percentTemplate],
    'L': ['HQM Score', percentTemplate],
}

for column in columnFormats.keys():
    writer.sheets['Momentum Strategy'].set_column(
        f'{column}:{column}',
        len(columnFormats[column][0]),
        columnFormats[column][1],
    )
    writer.sheets['Momentum Strategy'].write(
        f'{column}1',
        columnFormats[column][0],
        columnFormats[column][1],
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [84]:
writer.save()